# Tarea 1 - Computación Cienctífica y Ciencia de los Datos
## Vectorización en Python: Generalización de autómatas de Conway
### Vicente Mieres.

## 1. Introducción

En el presente informe, se detalla la implmentacion de un generalización del juego de la vida de Conway, matriz infita con valores 0 o 1 por celda, las cuales "evolucionan" con el paso del tiempo. En particular para este trabajo, se debe extrapolar a grillas de n-dimensiones, con m posibles valores por celda. Además, este debe ser eficiente, por lo que la utilización de librerías como numpy y, sobretodo, cupy, son un requisito fundamental.

De esta forma, el objetivo de esta entrega, corresponde a la implementacion *eficiente* de un simulador, evitando el uso de ciclos tradicionales (ciclos *for* explícitos) y utilizando vectorización para trabajar de forma paralela. Además, se implmentará la version iterativa del problema, esto con el fin de realizar una comparativa a nivel de tiempo de ejecución, y asi demostrar la eficiencia de trabajar con programación paralela. Finalmente, se presentan diferentes ejemplos de la ejecución del código, para distintas combinaciones de parámetros.

## 2. Desarrollo

## 1. Librerías
En este apartado se definen las diferentes librería utilizadas para la correctam implementación del programa.
- **NumPy**: Encargada de la creación y manipulación de arreglos multidimensionales, operaciones vectorizadas, entre otros.
- **CuPy**: Implementacion de NumPy compatible con GPU. Permite acelerar el calculo de arreglos grandes utilizando computación paralela.
- **matplotlib.pyplot**: Biblioteca de visualización. Creación de graficas.
- **SciPy.ndimage.convolve**: Función específica de SciPy para el procesamiento de imagenes (Explicación más adelante).
- **matplotlib.use('TkAgg')**: Configura el backend de Matplotlib para creación de ventanas externas e interacción con gráficas.

In [ ]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from scipy.ndimage import convolve

import matplotlib
matplotlib.use('TkAgg')  # Usar TkAgg backend para ventanas externas

## 2. Implementación del Simulador

En la siguiente sección se especifican las funciones utilizadas para dar solución al problema planteado.

In [ ]:
def expand_grid(grid, radius):
    """
    Expands the grid with periodic boundaries (toroid).
    
    Parameters:
    - grid: NumPy ndarray.
    - radius: Neighborhood radius that determines how much to expand.
    
    Returns:
    - expanded_grid: Grid with periodic boundaries.
    """
    return np.pad(grid, radius, mode='wrap')

def get_neighbor_sum(expanded_grid, kernel):
    """
    Calculates the sum of values in the neighborhood for each cell.
    
    Parameters:
    - expanded_grid: Grid with expanded borders.
    - kernel: Kernel that defines the neighborhood.
    
    Returns:
    - neighbor_sum: Array with the sum of neighboring values for each cell.
    """
    # Convolution to obtain the sum of neighbors
    neighbor_sum = convolve(expanded_grid, kernel, mode='wrap')
    # Extract the valid part of the convolution
    radius = kernel.shape[0] // 2
    slices = tuple(slice(radius, -radius) for _ in range(expanded_grid.ndim))
    neighbor_sum = neighbor_sum[slices]
    
    return neighbor_sum

def get_intervals(kernel, m):
   """
   Calculates the intervals for applying the evolution rules.
   
   Parameters:
   - kernel: Kernel that defines the neighborhood.
   - m: Maximum number of states per cell.
   
   Returns:
   - intervals: Array with the limits of the 3 intervals.
   """
   num_neighbors = np.sum(kernel)
   SM = m * num_neighbors
   intervals = np.linspace(0, SM, 4)  # 3 intervals
   return intervals

def evolve_grid(grid, neighbor_sum, intervals, m):
   """
   Applies evolution rules to the cellular automaton.
   
   Parameters:
   - grid: Current configuration of the automaton.
   - neighbor_sum: Sum of values in the neighborhood for each cell.
   - intervals: Limits of the intervals for applying rules.
   - m: Maximum number of states per cell.
   
   Returns:
   - new_grid: Grid evolved according to the rules.
   """
   # Copy of the grid to avoid modifying the original
   new_grid = grid.copy()
   
   # Apply rules with masks
   mask1 = (neighbor_sum >= intervals[0]) & (neighbor_sum < intervals[1])
   mask2 = (neighbor_sum >= intervals[1]) & (neighbor_sum < intervals[2])
   mask3 = (neighbor_sum >= intervals[2])
   
   new_grid[mask1] = np.maximum(new_grid[mask1] - 1, 0)  # Avoid negatives
   new_grid[mask2] = np.minimum(new_grid[mask2] + 1, m)  # Does not exceed m
   new_grid[mask3] = np.maximum(new_grid[mask3] - 1, 0)  # Avoid negatives
   return new_grid

## 3. Funciones Auxiliares

A continuación se presentan funciones auxiliares, estas  se utilizan para creación de grillas con patrones específicos, o visualización de la mismas para N dimensiones.

In [2]:
def create_grid(d, grid_size, m, pattern='random'):
    """
    Create a n-dimensional grid of a specified pattern.
   
    Parameters:
    - d: number of dimensions
    - grid_size: size of each dimension. (Array lenght)
    - m: max value for each cell
    - pattern: type of initial pattern ('random', 'central', 'checkerboard', 'cross', 'line')

    Return:
    - grid: n-dimensional grid
    """
    
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
   
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return np.random.randint(0, m+1, grid_size)
   
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
       
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
   
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = np.indices(grid_size)
        sum_indices = np.sum(indices, axis=0)
        return (sum_indices % 2) * m
        
    elif pattern == 'cross':
        # Patrón de cruz
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Para cada dimensión, crear una línea a través del centro
        for dim in range(d):
            # Crear índices para la línea en esta dimensión
            indices = [slice(0, s) if i != dim else center[i] for i, s in enumerate(grid_size)]
            grid[tuple(indices)] = m
        
        return grid
        
    elif pattern == 'line':
        # Patrón de línea horizontal 
        grid = np.zeros(grid_size, dtype=int)
    
        # Para una línea horizontal, necesitamos fijar la coordenada y y variar x
        if d >= 2:
            indices = [grid_size[1] // 2, slice(0, grid_size[0])]
            # Para dimensiones adicionales, elegimos el punto central
            for dim in range(2, d):
                indices.append(grid_size[dim] // 2)
            grid[tuple(indices)] = m
        else:  # Para el caso unidimensional
            grid[:] = m
        
        return grid
   
    else:
        # Patrón predeterminado: valores 0
        return np.zeros(grid_size, dtype=int)

In [3]:
def euclidean_kernel(dimensiones, radio):
    """
    Build an euclidean kernel. Circle shape in 2D, spheric shape in 3D, etc.
    
    Parámetros:
    - dimensiones: Number of dimensions.
    - radio: Radius of neighbours.
    
    Return:
    - kernel: Numpy array of 1s where there is neighbourhood and 0s where there's not.
    """
    # Construir el kernel para una vecindad euclidiana
    kernel_shape = (2 * radio + 1,) * dimensiones
    kernel_size = np.prod(kernel_shape)
    
    # Usar reshape para generar las coordenadas de manera eficiente
    linear_indices = np.arange(kernel_size).reshape(kernel_shape)
    
    # Calcular coordenadas centrales
    center_coords = np.array([radio] * dimensiones)
    
    # Convertir índices lineales a coordenadas
    coords = np.unravel_index(linear_indices.flatten(), kernel_shape)
    coords = np.array(coords).reshape(dimensiones, -1).T
    
    # Calcular distancias al cuadrado al centro
    squared_distances = np.sum((coords - center_coords)**2, axis=1)
    
    # Crear kernel con forma euclidiana usando reshape
    kernel = (squared_distances <= radio**2).astype(np.int32)
    kernel = kernel.reshape(kernel_shape)
    
    # Localizar y excluir la celda central
    center_index = tuple(radio for _ in range(dimensiones))
    kernel[center_index] = 0
    
    return kernel

In [ ]:
def visualize(grid, m, r, iterations=5, pause=1, cmap='viridis'):
    """
    Simulate and visualize the evolution of the cellular automaton in sequential windows.
   
    Parámetros:
    - grid: 2D initial grid
    - m: max value for each cell
    - r: neighborhood radius
    - iterations: Number of iterations
    - pause: time between iterations (seconds)
    - cmap: color map
    
    """

    # Deactivate interactive mode
    plt.ioff()
    actual_grid = grid.copy()
   
    # history of generated grids
    history = [actual_grid.copy()]

    kernel = euclidean_kernel(grid.ndim, r)
   
   # Simulation
    for step in range(iterations):
        expanded_grid = expand_grid(actual_grid, r)
        neighbor_sum = get_neighbor_sum(expanded_grid, kernel)
        intervals = get_intervals(kernel, m)
        actual_grid = evolve_grid(actual_grid, neighbor_sum, intervals, m)
        history.append(actual_grid.copy())
   
    # show every grid
    for step, grid in enumerate(history):
        fig = plt.figure(figsize=(8, 6), num=f"Cellular Automaton - Step {step}")
        plt.imshow(grid, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Cell State')
        plt.title(f"Step {step}" + (" (Initial)" if step == 0 else ""))
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
       
        # Every 1 second (default) shows an image of the grid.
        if step < len(history) - 1:
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(pause)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()
   

In [14]:
def visualize_nd(initial_grid, m, r, iterations=5, pause=1, cmap='viridis', visualization_axes=(0, 1), fixed_values=None):
    """
    Simulate and visualize the evolution of the cellular automaton in higher dimensions.
    Shows 2D slices keeping the other dimensions fixed.
    
    Parameters:
    - initial_grid: n-dimensional initial grid
    - m: Maximum state value
    - r: Neighborhood radius
    - iterations: Number of iterations to simulate
    - pause: Time between iterations (seconds)
    - cmap: Color map
    - visualization_axes: Tuple with the two axes to visualize (default (0,1))
    - fixed_values: Dictionary with fixed values for other dimensions {dim: value}
                   If None, the midpoint of each non-visualized dimension will be used
    
    """
    
    # Deactivate interactive mode
    plt.ioff()
    
    # Start simulation
    current_grid = initial_grid.copy()
    
    # history of generated grid
    history = [current_grid.copy()]
    
    kernel = euclidean_kernel(initial_grid.ndim, r)
    
    # Simulation
    for step in range(iterations):
        expanded_grid = expand_grid(current_grid, r)
        neighbor_sum = get_neighbor_sum(expanded_grid, kernel)
        intervals = get_intervals(kernel, m)
        current_grid = evolve_grid(current_grid, neighbor_sum, intervals, m)
        history.append(current_grid.copy())
    
    # Determine fixed values for non-visualized dimensions
    if fixed_values is None:
        fixed_values = {}
        
    # Get grid dimensions
    dimensions = initial_grid.shape
    ndim = len(dimensions)
    
    # Verify that visualization_axes are valid
    axis_x, axis_y = visualization_axes
    if axis_x >= ndim or axis_y >= ndim or axis_x < 0 or axis_y < 0 or axis_x == axis_y:
        raise ValueError(f"Visualization axes must be different and less than {ndim}")
    
    # Set default values for non-visualized dimensions
    for dim in range(ndim):
        if dim != axis_x and dim != axis_y and dim not in fixed_values:
            fixed_values[dim] = dimensions[dim] // 2  # Default midpoint
    
    # Show each grid sequentially
    for step, grid in enumerate(history):
        fig = plt.figure(figsize=(8, 6), num=f"Cellular Automaton {ndim}D - Step {step}")
        
        # Extract the 2D slice for visualization
        indices = [slice(None) if i == axis_x or i == axis_y else fixed_values[i] for i in range(ndim)]
        slice_2d = grid[tuple(indices)]
        
        # Show the 2D slice
        plt.imshow(slice_2d, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Cell State')
        
        # Create title with slice information
        slice_info = ", ".join([f"dim{i}={fixed_values[i]}" for i in range(ndim)
                               if i != axis_x and i != axis_y])
        plt.title(f"Step {step} ({slice_info})" + (" (Initial)" if step == 0 else ""))
        
        plt.xlabel(f"Dimension {axis_x}")
        plt.ylabel(f"Dimension {axis_y}")
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
        
        if step < len(history) - 1:
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(pause)
            plt.close()
        else:
            # For the last window, show without blocking
            plt.tight_layout()
            plt.show()

## 4. Ejemplos

In [9]:
r = 2
m = 1
griz_size = 10
d = 2
grid = create_grid(d, griz_size, m, pattern='cross')


In [10]:
# visualize(grid, m ,r, iterations=30)

In [12]:
# visualizar_nd(grid, m ,r, iteraciones=30)